In [2]:
import piheaan
import os

In [6]:
params = piheaan.ParameterPreset.FGb 
context = piheaan.make_context(params) 
piheaan.make_bootstrappable(context) 

# parameter 정보는 https://heaan.it/ 에서 확인 가능

In [8]:
#키생성
sk = piheaan.SecretKey(context) 
key_file_path = "./key" 
os.makedirs(key_file_path, mode=0o775, exist_ok=True) 
log_num_slot = piheaan.get_log_full_slots(context) 
num_slot = 1 << log_num_slot 
sk.save(key_file_path+"/secretkey.bin") 
sk = piheaan.SecretKey(context,key_file_path+"/secretkey.bin") 
key_generator = piheaan.KeyGenerator(context, sk) 
key_generator.gen_common_keys() 
key_generator.save(key_file_path + "/")

In [9]:

#생성된 키 받아오기
sk = piheaan.SecretKey(context) 
key_file_path = "./key" 
log_num_slot = piheaan.get_log_full_slots(context) 
num_slot = 1 << log_num_slot 
sk = piheaan.SecretKey(context,key_file_path+"/secretkey.bin")

keypack = piheaan.KeyPack(context, key_file_path+"/") 
keypack.load_enc_key() 
keypack.load_mult_key() 
eval = piheaan.HomEvaluator(context,keypack) 
dec = piheaan.Decryptor(context) 
enc = piheaan.Encryptor(context)

In [19]:
#1로 가득 찬 암호문 만들기
one_ = [1]*num_slot
one_msg = piheaan.Message(log_num_slot)
for k in range(num_slot):
    one_msg[k] = one_[k]

one_ctxt = piheaan.Ciphertext(context)
enc.encrypt(one_msg, keypack, one_ctxt)

#2로 가득 찬 암호문 만들기
two_ = [2]*num_slot
two_msg = piheaan.Message(log_num_slot)
for k in range(num_slot):
    two_msg[k] = two_[k]

two_ctxt = piheaan.Ciphertext(context)
enc.encrypt(two_msg, keypack, two_ctxt)

In [20]:
#1+2
result_add = piheaan.Ciphertext(context)
eval.add(one_ctxt, two_ctxt, result_add)

add_msg = piheaan.Message(log_num_slot)
dec.decrypt(result_add,sk,add_msg)

for i in range(0,5):
	print(add_msg[i])

(3+0j)
(3+0j)
(3+0j)
(3+0j)
(3+0j)


In [22]:
#1*2
result_mult = piheaan.Ciphertext(context)
eval.mult(one_ctxt, two_ctxt, result_mult)

mult_msg = piheaan.Message(log_num_slot)
dec.decrypt(result_mult,sk,mult_msg)

for i in range(0,5):
	print(mult_msg[i])

(2+0j)
(2+0j)
(2+0j)
(2+0j)
(2+0j)
